In [1]:
users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

In [2]:
# Recommending the most popular

from collections import Counter

popular_interests = Counter(interest
                            for user_interests in users_interests
                            for interest in user_interests)

popular_interests.most_common()

[('Python', 4),
 ('R', 4),
 ('Big Data', 3),
 ('HBase', 3),
 ('Java', 3),
 ('statistics', 3),
 ('regression', 3),
 ('probability', 3),
 ('Hadoop', 2),
 ('Cassandra', 2),
 ('MongoDB', 2),
 ('Postgres', 2),
 ('scikit-learn', 2),
 ('statsmodels', 2),
 ('pandas', 2),
 ('machine learning', 2),
 ('libsvm', 2),
 ('C++', 2),
 ('neural networks', 2),
 ('deep learning', 2),
 ('artificial intelligence', 2),
 ('Spark', 1),
 ('Storm', 1),
 ('NoSQL', 1),
 ('scipy', 1),
 ('numpy', 1),
 ('decision trees', 1),
 ('Haskell', 1),
 ('programming languages', 1),
 ('mathematics', 1),
 ('theory', 1),
 ('Mahout', 1),
 ('MapReduce', 1),
 ('databases', 1),
 ('MySQL', 1),
 ('support vector machines', 1)]

In [3]:
from typing import List, Tuple

def most_popular_new_interests(
    user_interests: List[str], 
    max_results: int = 5) -> List[Tuple[str, int]]:
    
    suggestions = [(interest, frequency) 
                   for interest, frequency in popular_interests.most_common() 
                   if interest not in user_interests]
    
    return suggestions[:max_results]

In [4]:
most_popular_new_interests(["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"])

[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

In [5]:
most_popular_new_interests(["R", "Python", "statistics", "regression", "probability"])

[('Big Data', 3), ('HBase', 3), ('Java', 3), ('Hadoop', 2), ('Cassandra', 2)]

In [6]:
# User-based Collaborative Filtering

unique_interests = sorted({interest
                           for user_interests in users_interests 
                           for interest in user_interests})

assert unique_interests[:6] == [
    'Big Data',
    'C++',
    'Cassandra',
    'HBase',
    'Hadoop',
    'Haskell'
]

In [7]:
# User feature

def make_user_interest_vector(user_interests: List[str]) -> List[int]:
    """
    Given a list of interests, produce a vector whose ith element is 1
    if unique_interests[i] is in the list, 0 otherwise
    """
    return [1 if interest in user_interests else 0
           for interest in unique_interests]

In [8]:
user_interest_vectors = [make_user_interest_vector(user_interests)
                         for user_interests in users_interests] # user-item matrix

In [9]:
# Pair-wise user similarities

from scratch.nlp import cosine_similarity

user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j) 
                      for interest_vector_j in user_interest_vectors] 
                     for interest_vector_i in user_interest_vectors]

100%|█████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1725.77it/s]


0 Java 3
0 Big Data 3
0 Hadoop 2
0 HBase 1
0 C++ 1
0 Spark 1
0 Storm 1
0 programming languages 1
0 MapReduce 1
0 Cassandra 1
0 deep learning 1
1 HBase 2
1 neural networks 2
1 Postgres 2
1 MongoDB 2
1 machine learning 2
1 Cassandra 1
1 numpy 1
1 decision trees 1
1 deep learning 1
1 databases 1
1 MySQL 1
1 NoSQL 1
1 artificial intelligence 1
1 scipy 1
2 regression 3
2 Python 2
2 R 2
2 libsvm 2
2 scikit-learn 2
2 mathematics 1
2 support vector machines 1
2 Haskell 1
2 Mahout 1
3 statistics 3
3 probability 3
3 Python 2
3 R 2
3 pandas 2
3 statsmodels 2
3 C++ 1
3 artificial intelligence 1
3 theory 1
['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra']
Big Data and programming languages 7

['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']
Python and statistics 5

['Python', 'scikit-learn', 'scipy', 'numpy', 'statsmodels', 'pandas']
Python and statistics 2
databases 2
machine learning 2

['R', 'Python', 'statistics', 'regression', 'probability']
machine learning 3
databa

<Figure size 432x288 with 0 Axes>

In [10]:
user_similarities[0]

[1.0,
 0.3380617018914066,
 0.0,
 0.0,
 0.0,
 0.1543033499620919,
 0.0,
 0.0,
 0.1889822365046136,
 0.5669467095138409,
 0.0,
 0.0,
 0.0,
 0.1690308509457033,
 0.0]

In [11]:
# Users 0 and 9 share interests in Hadoop, Java, and Big Data
assert 0.56 < user_similarities[0][9] < 0.58, "several shared interests"

# Users 0 and 8 share only one interest: Big Data
assert 0.18 < user_similarities[0][8] < 0.20, "only one shared interest"

In [12]:
def most_similar_users_to(user_id: int) -> List[Tuple[int, float]]:
    pairs = [(other_user_id, similarity) 
             for other_user_id, similarity in enumerate(user_similarities[user_id]) 
             if user_id != other_user_id and similarity > 0]
    
    return sorted(pairs, 
                  key=lambda pair: pair[-1], 
                  reverse=True)

In [13]:
most_similar_users_to(0)

[(9, 0.5669467095138409),
 (1, 0.3380617018914066),
 (8, 0.1889822365046136),
 (13, 0.1690308509457033),
 (5, 0.1543033499620919)]

In [14]:
from collections import defaultdict

def user_based_suggestions(user_id: int, 
                           include_current_interests: bool = False):
    
    # Sum up the user similarities for each interest
    suggestions: Dict[str, float] = defaultdict(float)
        
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity # more weight for a specific topic interested by users who similar to the current user
            
    # Convert them to a sorted list
    suggestions = sorted(suggestions.items(), 
                         key=lambda pair: pair[-1], 
                         reverse=True)
    
    # And exclude already interests
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight) 
                for suggestion, weight in suggestions 
                if suggestion not in users_interests[user_id]]

In [15]:
user_based_suggestions(0)

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

In [16]:
# Item-based Collaborative Filtering

interest_user_matrix = [[user_interest_vector[j] 
                         for user_interest_vector in user_interest_vectors] 
                        for j, _ in enumerate(unique_interests)] # item-user matrix

In [17]:
interest_user_matrix # row: interests, column: users

[[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,

In [18]:
# Pair-wise item similarities

interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j) 
                          for user_vector_j in interest_user_matrix] 
                         for user_vector_i in interest_user_matrix]

In [19]:
interest_similarities[0]

[1.0,
 0.0,
 0.4082482904638631,
 0.3333333333333333,
 0.8164965809277261,
 0.0,
 0.6666666666666666,
 0.0,
 0.5773502691896258,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5773502691896258,
 0.5773502691896258,
 0.4082482904638631,
 0.0,
 0.0,
 0.4082482904638631,
 0.0,
 0.0,
 0.0,
 0.4082482904638631,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [20]:
def most_similar_interests_to(interest_id: int):
    similarities = interest_similarities[interest_id]
    
    pairs = [(unique_interests[other_interest_id], similarity) 
             for other_interest_id, similarity in enumerate(similarities) 
             if interest_id != other_interest_id and similarity > 0]
    
    return sorted(pairs, 
                  key=lambda pair: pair[-1], 
                  reverse=True)

In [21]:
most_similar_interests_to(0) # Big Data

[('Hadoop', 0.8164965809277261),
 ('Java', 0.6666666666666666),
 ('MapReduce', 0.5773502691896258),
 ('Spark', 0.5773502691896258),
 ('Storm', 0.5773502691896258),
 ('Cassandra', 0.4082482904638631),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('HBase', 0.3333333333333333)]

In [22]:
def item_based_suggestions(user_id: int, 
                           include_current_interests: bool = False):
    
    # Add up the similar interests
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_vectors[user_id]
    
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1: # a specific topic interested by this user
            similar_interests = most_similar_interests_to(interest_id)
            
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity
                
    # Sort them by weight
    suggestions = sorted(suggestions.items(), 
                         key=lambda pair: pair[-1], 
                         reverse=True)
    
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight) 
                for suggestion, weight in suggestions 
                if suggestion not in users_interests[user_id]]

In [23]:
item_based_suggestions(0)

[('MapReduce', 1.861807319565799),
 ('MongoDB', 1.3164965809277263),
 ('Postgres', 1.3164965809277263),
 ('NoSQL', 1.2844570503761732),
 ('MySQL', 0.5773502691896258),
 ('databases', 0.5773502691896258),
 ('Haskell', 0.5773502691896258),
 ('programming languages', 0.5773502691896258),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('C++', 0.4082482904638631),
 ('Python', 0.2886751345948129),
 ('R', 0.2886751345948129)]

In [24]:
# Matrix Factorization

MOVIES = "data\\ml-100k\\u.item"  # pipe-delimited: movie_id|title|...
RATINGS = "data\\ml-100k\\u.data" # tab-delimited: user_id, movie_id, rating, timestamp

from typing import NamedTuple

class Rating(NamedTuple):
    user_id: str
    movie_id: str
    rating: float

In [25]:
import csv

with open(MOVIES, encoding="iso-8859-1") as f:
    reader = csv.reader(f, delimiter="|")
    movies = {movie_id: title for movie_id, title, *_ in reader}
    
with open(RATINGS, encoding="iso-8859-1") as f:
    reader = csv.reader(f, delimiter="\t")
    ratings = [Rating(user_id, movie_id, float(rating)) 
               for user_id, movie_id, rating, _ in reader]
    
assert len(movies) == 1682
assert len(list({rating.user_id for rating in ratings})) == 943

In [26]:
import re

# Data structure for accumulating ratings by movie_id
star_wars_ratings = {movie_id: [] 
                     for movie_id, title in movies.items() 
                     if re.search("Star Wars|Empire Strikes|Jedi", title)}

# Iterate over ratings, accumulating the Star Wars ones
for rating in ratings:
    if rating.movie_id in star_wars_ratings:
        star_wars_ratings[rating.movie_id].append(rating.rating)
        
# Compute the average rating for each movie
avg_ratings = [(sum(title_ratings) / len(title_ratings), movie_id) 
               for movie_id, title_ratings 
               in star_wars_ratings.items()]

# And then print them in order
for avg_rating, movie_id in sorted(avg_ratings, reverse=True):
    print(f"{avg_rating:.2f} {movies[movie_id]}")

4.36 Star Wars (1977)
4.20 Empire Strikes Back, The (1980)
4.01 Return of the Jedi (1983)


In [27]:
import random
random.seed(0)
random.shuffle(ratings)

split1 = int(len(ratings) * 0.7)
split2 = int(len(ratings) * 0.85)

train = ratings[:split1]            # 70% train
validation = ratings[split1:split2] # 15% validation
test = ratings[split2:]             # 15% test

In [28]:
# Baseline model

avg_rating = sum(rating.rating for rating in train) / len(train)
baseline_error = sum((rating.rating - avg_rating) ** 2 
                     for rating in test) / len(test)

assert 1.26 < baseline_error < 1.27

In [29]:
# User and item embeddings

from scratch.deep_learning import random_tensor

EMBEDDING_DIM = 2

# Find unique ids
user_ids = {rating.user_id for rating in ratings}
movie_ids = {rating.movie_id for rating in ratings}

# Then create a random vector per id
user_vectors = {user_id: random_tensor(EMBEDDING_DIM) 
                for user_id in user_ids}
movie_vectors = {movie_id: random_tensor(EMBEDDING_DIM) 
                 for movie_id in movie_ids}

In [30]:
from typing import List
import tqdm
from scratch.linear_algebra import dot

def loop(dataset: List[Rating],
         learning_rate: float = None) -> None:
    
    with tqdm.tqdm(dataset) as t:
        loss = 0.0

        for i, rating in enumerate(t):
            movie_vector = movie_vectors[rating.movie_id]
            user_vector = user_vectors[rating.user_id]
            predicted = dot(user_vector, movie_vector)
            error = predicted - rating.rating
            loss += error ** 2

            if learning_rate is not None:
                #     predicted = m_0 * u_0 + ... + m_k * u_k
                # So each u_j enters output with coefficent m_j
                # and each m_j enters output with coefficient u_j
                user_gradient = [error * m_j for m_j in movie_vector]
                movie_gradient = [error * u_j for u_j in user_vector]

                # Take gradient steps
                for j in range(EMBEDDING_DIM):
                    user_vector[j] -= learning_rate * user_gradient[j]
                    movie_vector[j] -= learning_rate * movie_gradient[j]

            t.set_description(f"avg loss: {loss / (i + 1)}")

In [31]:
learning_rate = 0.05
for epoch in range(20):
    learning_rate *= 0.9
    print(epoch, learning_rate)
    loop(train, learning_rate=learning_rate)
    loop(validation)
loop(test)

avg loss: 15.107575888126913:   0%|                                           | 144/70000 [00:00<00:48, 1429.55it/s]

0 0.045000000000000005


avg loss: 1.039478084619848:   0%|                                            | 101/70000 [00:00<01:09, 1002.64it/s]

2 0.03645000000000001


avg loss: 0.9375644393738976:   0%|                                           | 120/70000 [00:00<00:58, 1197.11it/s]

...

avg loss: 0.8308758294856923:   0%|                                             | 93/70000 [00:00<01:16, 914.20it/s]

18 0.006754258588364966


avg loss: 0.7807713825637939:   0%|                                           | 126/70000 [00:00<00:55, 1250.86it/s]

19 0.00607883272952847


avg loss: 0.9078059874789042:  98%|█████████████████████████████████████████ | 14662/15000 [00:16<00:00, 729.56it/s]